In [2]:
def GetPlayerLogs(playername, seasons_played: list):
    from nba_api.stats.endpoints import PlayerGameLog
    from nba_api.stats.static import players
    import pandas as pd

    # Step 1: Find Player ID
    def get_player_id(player_name):
        nba_players = players.get_players()  # Get all NBA players
        for player in nba_players:
            if player['full_name'].lower() == player_name.lower():
                return player['id']
        return None

    # Get Player's ID
    player_id = get_player_id(playername)
    if not player_id:
        print("Player not found.")
        return

    # Step 2: Retrieve Game Logs for the Seasons
    def get_player_game_logs(player_id, season):
        try:
            game_log = PlayerGameLog(player_id=player_id, season=season)
            game_log_data = game_log.get_data_frames()[0]  # Get the first DataFrame
            return game_log_data
        except Exception as e:
            print(f"Error fetching game logs for season {season}: {e}")
            return pd.DataFrame()  # Return an empty DataFrame if an error occurs

    # Fetch the Player's game logs for the specified seasons
    game_logs = [get_player_game_logs(player_id, season) for season in seasons_played]

    # Combine all DataFrames in the list into one DataFrame
    combined_logs = pd.concat(game_logs, ignore_index=True) if game_logs else pd.DataFrame()

    # Step 3: Save the Game Logs to a CSV File
    if not combined_logs.empty:
        min_season = min(seasons_played)
        max_season = max(seasons_played)
        if max_season == min_season:
            filename = f"{playername}_{max_season}_game_logs.csv"
        else:
            filename = f"{playername}_{min_season}_to_{max_season}_game_logs.csv"
        combined_logs.to_csv(filename, index=False)
        print(f"Game logs saved as {filename}")
    else:
        print("No game logs found for the specified seasons.")


In [3]:
GetPlayerLogs('Devin Booker',['2022-23'])

Game logs saved as Devin Booker_2022-23_game_logs.csv
